In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchvision.models import Inception_V3_Weights
import torch.nn.functional as F

# ----------------------------
# CONFIGURATION
# ----------------------------
model_type = 'custom'  # Options: 'custom', 'alexnet', 'resnet', 'inception'
num_epochs = 5
batch_size = 64
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# DATA TRANSFORMS
# ----------------------------
if model_type == 'inception':
    input_size = 299
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
else:
    input_size = 32
    normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                     std=[0.5, 0.5, 0.5])

transform = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    normalize
])

# ----------------------------
# DATA LOADERS
# ----------------------------
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ----------------------------
# MODEL DEFINITIONS
# ----------------------------

class CustomCNN(nn.Module):  # Task E
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(256*4*4, 512), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        return self.net(x)

def get_model(name):
    if name == 'custom':
        return CustomCNN()
    elif name == 'alexnet':  # Task B
        model = models.alexnet(pretrained=True)
        model.classifier[6] = nn.Linear(4096, 10)
        return model
    elif name == 'resnet':  # Task C
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, 10)
        return model
    elif name == 'inception':  # Task D
        weights = Inception_V3_Weights.DEFAULT
        model = models.inception_v3(weights=weights)
        model.fc = nn.Linear(model.fc.in_features, 10)
        return model
    else:
        raise ValueError("Invalid model type!")

# ----------------------------
# INITIALIZE MODEL
# ----------------------------
model = get_model(model_type).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ----------------------------
# TRAINING
# ----------------------------
print(f"\nTraining {model_type.upper()} on CIFAR-10...\n")

model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        if model_type == 'inception':
            outputs, aux_outputs = model(images)
            loss1 = criterion(outputs, labels)
            loss2 = criterion(aux_outputs, labels)
            loss = loss1 + 0.4 * loss2
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {avg_loss:.4f}")

# ----------------------------
# EVALUATION
# ----------------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"\nTest Accuracy of {model_type.upper()} on CIFAR-10: {accuracy:.2f}%")



Training CUSTOM on CIFAR-10...

Epoch [1/5] - Loss: 1.4808
Epoch [2/5] - Loss: 1.0928
Epoch [3/5] - Loss: 0.9336
Epoch [4/5] - Loss: 0.8191
Epoch [5/5] - Loss: 0.7395

Test Accuracy of CUSTOM on CIFAR-10: 76.79%
